## **Exercise 1.05 - Guided Exercise**
### Outliers - Quantity Investigation

### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

%matplotlib inline

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import certifi
from sklearn.datasets import fetch_openml

### Loading the Data

In [ ]:

df = pd.read_csv('https://raw.githubusercontent.com/fenago/datawrangling/main/miami-housing.csv')
df.sample(5)
df.head()

## **Structure Investigation**



In [ ]:
# Show size of the dataset
df.shape


In [ ]:
# Count how many times each data type is present in the dataset
pd.value_counts(df.dtypes)

## Structure of numerical features


In [ ]:
# For each numerical feature compute number of unique entries
unique_values = df.select_dtypes(include='number').nunique().sort_values()
plt.figure(figsize=(15, 4))
sns.set_style('whitegrid')


In [ ]:

g = sns.barplot(x=unique_values.index, y=unique_values, palette='inferno')
g.set_yscale("log")
g.set_xticklabels(g.get_xticklabels(), rotation=45, horizontalalignment='right')
g.set_title('Unique values per frequency')
plt.show()

##  Conclusion of structure investigation


In [ ]:
df.info() 
df.describe()

## Quality Investigation
Duplicates, missing values and unwanted entries or errors will be seen


In [ ]:
# Check number of duplicates while ignoring the index feature
n_duplicates = df.drop(labels=['PARCELNO'], axis=1).duplicated().sum()

print(f"You seem to have {n_duplicates} duplicates in your database.")

In [ ]:
#  Extract column names of all features, except 'PARCELNO'
columns_to_consider = df.drop(labels=['PARCELNO'], axis=1).columns

# Drop duplicates based on 'columns_to_consider'
df.drop_duplicates(subset=columns_to_consider, inplace=True)
df.shape

In [ ]:
plt.figure(figsize=(15, 4))
sns.set_style('whitegrid')

g = sns.barplot(x=unique_values.index, y=unique_values, palette='inferno')
g.set_yscale("log")
g.set_xticklabels(g.get_xticklabels(), rotation=45, horizontalalignment='right')
g.set_title('Unique values per frequency')
plt.show()

In [ ]:

plt.figure(figsize=(15, 8))
sns.set_style('whitegrid')

g = sns.heatmap(df_X.isnull(), cbar=False, cmap='viridis')
g.set_xlabel('Column Number')
g.set_ylabel('Sample Number')

In [ ]:
!pip install missingno
import missingno as msno
msno.matrix(df, labels=True, sort='descending', color=(0.27, 0.52, 1.0));
# g = msno.bar(df_X, labels=True, color="dodgerblue", sort="ascending", figsize=(10,5), fontsize=12)
# g.set_xticklabels(g.get_xticklabels(),rotation=90);

In [ ]:

df = df.dropna(thresh=df.shape[1] * 0.80, axis=0).reset_index(drop=True)
df.shape

##  Per Feature
Let's look at the number of missing values per feature. 


In [ ]:
df.isna().mean().sort_values().plot(
    kind="bar", figsize=(15, 4),
    title="Percentage of missing values per feature",
    ylabel="Ratio of missing values per feature");


In [ ]:

df = df.dropna(thresh=df.shape[0] * 0.85, axis=1)
df.shape

##  Numerical Features
 Pandas' .plot() function will be used here:

In [ ]:
df.plot(lw=0, marker=".", subplots=True, layout=(-1, 4),
          figsize=(15, 30), markersize=1);

##  Non-numerical features

In [ ]:
# Display non-numerical features
df.select_dtypes(exclude="number").head()


### **Content Investigation**

### Feature Distribution

In [ ]:
# Plots the histogram for each numerical feature in a separate subplot
df.hist(bins=25, figsize=(15, 25), layout=(-1, 5), edgecolor="black")
plt.tight_layout();

In [ ]:
# Collects for each feature the most frequent entry
# most_frequent_entry = df_X.mode()

# Checks for each entry if it contains the most frequent entry
# df_freq = df_X.eq(most_frequent_entry.values)

# Computes the mean of the 'is_most_frequent' occurrence
# df_freq = df_freq.mean().sort_values(ascending=False)

# Show the 5 top features with the highest ratio of singular value content
# display(df_freq.head())

# Visualize the 'df_freq' table
# df_freq.plot.bar(figsize=(15, 4));

### Patterns

### Continuous Features

In [ ]:

# Creates mask to identify numerical features with more or less than 25 unique features
cols_continuous = df.select_dtypes(include="number").nunique() >= 25

# Create a new dataframe which only contains the continuous features
df_continuous = df[cols_continuous[cols_continuous].index]
df_continuous.shape

sns.pairplot(df_continuous, height=1.5, plot_kws={"s": 2, "alpha": 0.2});

In [ ]:
# Create a new dataframe which doesn't contain the numerical continuous features
df_discrete = df[cols_continuous[~cols_continuous].index]
df_discrete.shape

In [ ]:


# Establish number of columns and rows needed to plot all features
# n_cols = 5
# n_elements = len(df_discrete.columns)
# n_rows = np.ceil(n_elements / n_cols).astype("int")

# Specify y_value to spread data (ideally a continuous feature)
# y_value = df_X["SALE_PRC"]

# Create figure object with as many rows and columns as needed
# fig, axes = plt.subplots(ncols=n_cols, nrows=n_rows, figsize=(15, n_rows * 2.5))

# Loop through features and put each subplot on a matplotlib axis object
# for col, ax in zip(df_discrete.columns, axes.ravel()):
#     sns.stripplot(data=df_X, x=col, y=y_value, ax=ax, palette="tab10", size=1, alpha=0.5)
# plt.tight_layout();


### Feature Relationships

In [ ]:

# Evaluate but remember to consider multicollinearity

# Computes feature correlation
df_corr = df.corr(method="pearson")

# Create labels for the correlation matrix
labels = np.where(np.abs(df_corr)>0.75, "S",
                  np.where(np.abs(df_corr)>0.5, "M",
                           np.where(np.abs(df_corr)>0.25, "W", "")))

# Plot correlation matrix
plt.figure(figsize=(15, 15))
sns.heatmap(df_corr, mask=np.eye(len(df_corr)), square=True,
            center=0, annot=labels, fmt='', linewidths=.5,
            cmap="vlag", cbar_kws={"shrink": 0.8});

In [ ]:
#  Creates a mask to remove the diagonal and the upper triangle.
lower_triangle_mask = np.tril(np.ones(df_corr.shape), k=-1).astype("bool")

#  Stack all correlations, after applying the mask
df_corr_stacked = df_corr.where(lower_triangle_mask).stack().sort_values()

#  Showing the lowest and highest correlations in the correlation matrix
display(df_corr_stacked)

## **Outliers**

In [ ]:
Q1 = df_X.quantile(0.25)
Q3 = df_X.quantile(0.75)
IQR = Q3 - Q1
print(IQR)
# print(df_X < (Q1 - 1.5 * IQR)) |(df_X > (Q3 + 1.5 * IQR))

In [ ]:

df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
df_out.shape